# Start

In [37]:
import importlib
import os
# import pandas as pd
# import json
# from datetime import datetime
from pathlib import Path
# from io import StringIO

from langchain_core.tools import Tool, StructuredTool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

from bsm_multi_agents.graph.state import WorkflowState

In [38]:
# from bsm_multi_agents.agents import utils
# importlib.reload(utils)
# from bsm_multi_agents.agents.utils import get_tool_result_from_messages,print_resp

In [39]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

WindowsPath('c:/Users/bobby/source/repos/OPA1118')

# Graph

In [ ]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_tool_node,
)

from bsm_multi_agents.agents.summary_generator_agent import (
    summary_generator_node,
)

from bsm_multi_agents.agents.report_generator_agent import (
    report_generator_node,
)

graph = StateGraph(WorkflowState)

graph.add_node("pricing_calculator_tool", pricing_calculator_tool_node)
graph.add_node("pricing_validator_tool", pricing_validator_tool_node)
graph.add_node("summary_generator_tool", summary_generator_node)
graph.add_node("report_generator_tool", report_generator_node)

graph.add_edge(START, "pricing_calculator_tool")
graph.add_edge("pricing_calculator_tool", "pricing_validator_tool")
# graph.add_edge("pricing_validator_tool", END)
graph.add_edge("pricing_validator_tool", "summary_generator_tool")
#raph.add_edge("summary_generator_tool", END)
graph.add_edge("summary_generator_tool", "report_generator_tool")
graph.add_edge("report_generator_tool", END)

app = graph.compile()

In [41]:
csv_file_path = "c:/Users/bobby/source/repos/OPA1118" + "/data/input/dummy_options.csv"#os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = "c:/Users/bobby/source/repos/OPA1118" + "/data/cache"#os.path.join(project_path, "data/cache")
server_path = "c:/Users/bobby/source/repos/OPA1118" + "/src/bsm_multi_agents/mcp/server.py"#os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
pricing_results_path = "c:/Users/bobby/source/repos/OPA1118" + "/data/cache/dummy_options_bsm_results.csv"#os.path.join(project_path, "data/cache/dummy_options_bsm_results.csv")
greeks_results_path = "c:/Users/bobby/source/repos/OPA1118" + "/data/cache/dummy_options_greeks_results.csv"#os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
validate_results_path = "c:/Users/bobby/source/repos/OPA1118" + "/data/cache/dummy_options_greeks_results_validate_results.csv"#os.path.join(project_path, "data/cache/dummy_options_greeks_results_validate_results.csv")
# md_summary_text = 
init_state = WorkflowState(
    csv_file_path=csv_file_path,
    output_dir=output_dir,
    server_path=server_path,
    pricing_results_path=pricing_results_path,
    greeks_results_path = greeks_results_path,
    validate_results_path = validate_results_path
    # "remaining_steps": 10,
    # "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
)

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

KeyboardInterrupt: 

In [ ]:
import pandas as pd
pd.read_csv(csv_file_path)

,date,S,K,T,r,sigma,option_type,asset_class
0,2025-09-01,1.124836,1.147031,0.396435,0.016240,2.500000,call,FX
1,2025-09-02,98.617357,105.839622,1.242119,0.038323,0.164521,call,Equity
2,2025-09-03,95.305256,89.270613,0.678060,0.030990,0.195456,call,Equity
3,2025-09-04,105.425600,101.023642,-0.100000,0.015580,0.257986,call,Equity
4,2025-09-05,1.054599,1.114748,0.479380,0.030569,0.095607,call,FX
...,...,...,...,...,...,...,...,...
195,2025-09-16,55.265764,59.074121,0.706291,0.043157,2.500000,put,Commodity
196,2025-09-17,100.961208,93.299603,0.246211,0.037852,0.207502,put,Equity
197,2025-09-18,95.377247,99.664774,0.224177,0.022612,0.234969,put,Equity
198,2025-09-19,1.172349,1.201867,1.783358,0.034635,0.081875,put,FX


In [ ]:
final_state

{'messages': [],
 'csv_file_path': 'c:/Users/bobby/source/repos/OPA1118/data/input/dummy_options.csv',
 'output_dir': 'c:/Users/bobby/source/repos/OPA1118/data/cache',
 'server_path': 'c:/Users/bobby/source/repos/OPA1118/src/bsm_multi_agents/mcp/server.py',
 'pricing_results_path': 'c:/Users/bobby/source/repos/OPA1118/data/cache/dummy_options_bsm_results.csv',
 'greeks_results_path': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_greeks_results.csv',
 'validate_results_path': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_greeks_results_validate_results.csv',
 'errors': [],
 'remaining_steps': 0}

# Calculator

## MCP Tool Level

### calculate_bsm_to_file

In [ ]:
%%time
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

CPU times: total: 0 ns
Wall time: 15 μs


In [ ]:
%%time
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_bsm_to_file

CPU times: total: 531 ms
Wall time: 518 ms


In [ ]:
%%time
calculate_bsm_to_file(csv_file_path, output_dir)

CPU times: total: 62.5 ms
Wall time: 36.7 ms


C:\Users\bobby\source\repos\OPA1118\src\bsm_multi_agents\mcp\pricing_calculator.py:11: RuntimeWarning: invalid value encountered in sqrt
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
C:\Users\bobby\source\repos\OPA1118\src\bsm_multi_agents\mcp\pricing_calculator.py:12: RuntimeWarning: invalid value encountered in sqrt
  d2 = d1 - sigma * np.sqrt(T)
C:\Users\bobby\source\repos\OPA1118\src\bsm_multi_agents\mcp\pricing_calculator.py:11: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


'c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_bsm_results.csv'

### calculate_greeks_to_file

In [ ]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_greeks_to_file

In [ ]:
calculate_greeks_to_file(csv_file_path, output_dir)

'c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_greeks_results.csv'

## MCP Server Level

## Call MCP Level

### call_mcp_tool_async

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import call_mcp_tool_async

In [ ]:
res = await call_mcp_tool_async(
    tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    arguments={
        "input_path": csv_file_path,
        "output_dir": output_dir,
    },
)
res

CallToolResult(meta=None, content=[TextContent(type='text', text='c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_bsm_results.csv', annotations=None, meta=None)], structuredContent={'result': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_bsm_results.csv'}, isError=False)

### call_mcp_tool

### run_in_new_loop

### create_mcp_state_tool_wrapper

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
current_state = {
    "csv_file_path": csv_file_path,
    "output_dir": output_dir
}

In [ ]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [ ]:
bsm_state_tool = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir"
    },
    output_key="bsm_results_path"
)

In [ ]:
result_update = bsm_state_tool(state=current_state)

In [ ]:
result_update

{'bsm_results_path': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_bsm_results.csv'}

## Agent Level

### calculator_tool_node

#### Node Inside Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [ ]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [ ]:
errors = current_state.get("errors", [])
[
    "csv_file_path" not in current_state or not current_state["csv_file_path"],
    "server_path" not in current_state or not current_state["server_path"]
]

[False, False]

In [ ]:
server_path = current_state["server_path"]
bsm_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="bsm_results_path",
)

greeks_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_greeks_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="greeks_results_path",
)

In [ ]:
bsm_update = bsm_state_fn(state=current_state)
greeks_update = greeks_state_fn(state=current_state)

In [ ]:
current_state["bsm_results_path"] = bsm_update["bsm_results_path"]
current_state["greeks_results_path"] = greeks_update["greeks_results_path"]

#### Node Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import pricing_calculator_tool_node

In [ ]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [ ]:
update_state = pricing_calculator_tool_node(state=current_state)
update_state

{'csv_file_path': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\data/input/dummy_options.csv',
 'output_dir': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\data/cache',
 'server_path': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\src/bsm_multi_agents/mcp/server.py',
 'bsm_results_path': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_bsm_results.csv',
 'greeks_results_path': 'c:\\Users\\bobby\\source\\repos\\OPA1118\\data\\cache\\dummy_options_greeks_results.csv',
 'errors': []}

### calculator_agent_node

#### Node Inside Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/calculator_prompts.txt")

In [ ]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

# from bsm_multi_agents.config.llm_config_claude import get_llm
from llm.llm_config_claude import get_llm
from bsm_multi_agents.prompts.loader import load_prompt

ModuleNotFoundError: No module named 'llm'

In [ ]:
current_state = WorkflowState(
        csv_file_path=csv_file_path, 
        output_dir=output_dir, 
        server_path=server_path
    )

In [ ]:
errors = current_state.get("errors", [])
"csv_file_path" not in current_state or not current_state["csv_file_path"]

False

In [ ]:
get_llm()

openai


ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x13a8ec440>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x13a8ecec0>, root_client=<openai.OpenAI object at 0x13a20bcb0>, root_async_client=<openai.AsyncOpenAI object at 0x13a8ecc20>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [ ]:
system_prompt = """
You are a quantitative calculator agent.
You have access to tools that call an external MCP server.
Each tool reads input file paths from the shared workflow state
and writes result file paths back into the state.
Do NOT ask for raw CSV content. Always work with paths only.
You do not need to provide any arguments to the tools. Just call them with {}.
"""
user_prompt_template = load_prompt(prompt_path)
formatted_user_prompt = (
    user_prompt_template.format(csv_file_path=current_state["csv_file_path"])
    if "{csv_file_path}" in user_prompt_template
    else user_prompt_template
)
llm = get_llm()
system_msg = SystemMessage(content=system_prompt)
human_msg = HumanMessage(content=formatted_user_prompt)
ai_msg = llm.invoke([system_msg, human_msg])

openai


In [ ]:
messages = list(current_state.get("messages", []))
messages.extend([system_msg, human_msg, ai_msg])
current_state["messages"] = messages

current_state["errors"] = errors

In [ ]:
current_state

{'csv_file_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache',
 'server_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\nYou are a quantitative calculator agent.\nYou have access to tools that call an external MCP server.\nEach tool reads input file paths from the shared workflow state\nand writes result file paths back into the state.\nDo NOT ask for raw CSV content. Always work with paths only.\nYou do not need to provide any arguments to the tools. Just call them with {}.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n-

#### Node Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import pricing_calculator_agent_node

from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

In [ ]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [ ]:
result_state = pricing_calculator_agent_node(current_state)

openai


In [ ]:
result_state

{'csv_file_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache',
 'server_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` 

### Combined Tool and Agent Levle

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

In [ ]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [ ]:
current_state = pricing_calculator_agent_node(current_state)
current_state = pricing_calculator_tool_node(current_state)

openai


In [ ]:
current_state

{'csv_file_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache',
 'server_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` 

# Validator

## MCP Tool Level

### validate_greeks_to_file

In [ ]:
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.mcp import pricing_validator
importlib.reload(pricing_validator)
from bsm_multi_agents.mcp.pricing_validator import validate_greeks_to_file

In [ ]:
validate_greeks_to_file(greeks_results_path, output_dir)

'/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv'

## Call MCP Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import call_mcp_tool_async

In [ ]:
res = await call_mcp_tool_async(
    tool_name="validate_greeks_to_file",
    server_script_path=server_path,
    arguments={
        "input_path": greeks_results_path,
        "output_dir": output_dir,
    },
)
res

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv', annotations=None, meta=None)], structuredContent={'result': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv'}, isError=False)

## Agent Level

### pricing_validator_agent_node

#### Node Inside Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [ ]:
state = WorkflowState(
    greeks_results_path=greeks_results_path, 
    output_dir=output_dir, 
    server_path=server_path
)


In [ ]:
errors = state.get("errors", [])
"greeks_results_path" not in state or not state["greeks_results_path"]

False

In [ ]:
server_path = state["server_path"]
validate_greeks_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="validate_greeks_to_file",
    server_script_path=server_path,
    input_arg_map={
        "greeks_results_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="validate_results_path",
)
validate_update = validate_greeks_state_fn(state=state)

In [ ]:
"errors" in validate_update

False

#### Node Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents import pricing_validator_node
importlib.reload(pricing_validator_node)
from bsm_multi_agents.agents.pricing_validator_node import pricing_validator_tool_node

In [ ]:
current_state = WorkflowState(
    greeks_results_path=greeks_results_path, 
    output_dir=output_dir, 
    server_path=server_path
)
update_state = pricing_validator_tool_node(state=current_state)
update_state

{'greeks_results_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
 'output_dir': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache',
 'server_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'validate_results_path': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv',
 'errors': []}

# Summary Documentation

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [ ]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import call_mcp_tool_async

In [ ]:
res = await call_mcp_tool_async(
    tool_name="csv_to_summary",
    server_script_path=server_path,
    arguments={
        "csv_path": greeks_results_path,
        "output_docx": output_dir + "/sample_output.docx",
    },
)
res

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/sample_output.docx', annotations=None, meta=None)], structuredContent={'result': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/sample_output.docx'}, isError=False)

# Word Documentation

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
output_dir = os.path.join(project_path, "data/cache")
summary_docx_path = os.path.join(project_path, "data/cache/sample_output.docx")
output_docx_path = os.path.join(project_path, "data/cache/final_OPA_report.docx")
model_name= "Option Procing Model with Black-Scholes model"
author_name= "John Doe"
group_name= "Front Desk Modeling and Analytics"
version= "v1.0"

In [ ]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import call_mcp_tool_async

In [ ]:
res = await call_mcp_tool_async(
    tool_name="summary_to_word",
    server_script_path=server_path,
    arguments={
        "summary_docx_path": summary_docx_path,
        "output_docx_path": output_docx_path,
        "model_name": model_name,
        "author_name": author_name,
        "group_name": group_name,
        "version": version,
    },
)
res

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/final_OPA_report.docx', annotations=None, meta=None)], structuredContent={'result': '/Users/zhenchenhong/Downloads/OPA/model_doc_automation/data/cache/final_OPA_report.docx'}, isError=False)